In [25]:
import pandas as pd


In [26]:
# 평일 / 주말 구분
data= {
    '일시': pd.date_range(start='2022-01-01', end='2024-01-01', freq='D'),
}
weekdays_df = pd.DataFrame(data)

# 주말 여부 판단(주말:1, 평일:0)
weekdays_df['주말'] = weekdays_df['일시'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

print(weekdays_df.head(10))

          일시  주말
0 2022-01-01   1
1 2022-01-02   1
2 2022-01-03   0
3 2022-01-04   0
4 2022-01-05   0
5 2022-01-06   0
6 2022-01-07   0
7 2022-01-08   1
8 2022-01-09   1
9 2022-01-10   0


In [27]:
# 열이름 확인
holidays_df = pd.read_csv('hd_22_24.csv')
print("공휴일 파일 열 이름:", holidays_df.columns)

공휴일 파일 열 이름: Index(['일시', 'dateName'], dtype='object')


In [28]:
# dateName -> 주말+공휴일
# 주말 데이터 + 공휴일

In [29]:
# 오른쪽의 dateName을 "공휴일"로 변경
if 'dateName' in holidays_df.columns:
    holidays_df = holidays_df.rename(columns={'dateName':'공휴일'})

holidays_df

,일시,공휴일
0,2022-01-01,1월1일
1,2022-01-31,설날
2,2022-02-01,설날
3,2022-02-02,설날
4,2022-03-01,삼일절
5,2022-03-09,대통령선거일
6,2022-05-05,어린이날
7,2022-05-08,부처님오신날
8,2022-06-06,현충일
9,2022-09-09,추석


In [34]:
# 주말 데이터 생성
weekday = {'일시': pd.date_range(start='2022-01-01', end='2024-10-30', freq='D')}
weekdays_df = pd.DataFrame(weekday)
weekdays_df['주말'] = weekdays_df['일시'].apply(lambda x: 1 if x.weekday() >= 5 else 0)
weekdays_df

,일시,주말
0,2022-01-01,1
1,2022-01-02,1
2,2022-01-03,0
3,2022-01-04,0
4,2022-01-05,0
...,...,...
1029,2024-10-26,1
1030,2024-10-27,1
1031,2024-10-28,0
1032,2024-10-29,0


In [35]:
# '일시' 열의 데이터 타입 확인
print(weekdays_df['일시'].dtype)
print(holidays_df['일시'].dtype)

# '일시' 열을 datetime으로 변환
holidays_df['일시'] = pd.to_datetime(holidays_df['일시'], errors='coerce')

datetime64[ns]
datetime64[ns]


In [39]:
weekdays_df

,일시,주말
0,2022-01-01,1
1,2022-01-02,1
2,2022-01-03,0
3,2022-01-04,0
4,2022-01-05,0
...,...,...
1029,2024-10-26,1
1030,2024-10-27,1
1031,2024-10-28,0
1032,2024-10-29,0


In [53]:
# '주말' 열을 정수형으로 변환
weekdays_df['주말'] = weekdays_df['주말'].astype(int)

# 주말 + 공휴일 데이터 병합
combined_df = pd.merge(weekdays_df, holidays_df, on='일시', how='left')

# 병합후 주말-> 주말_공휴일 이름변경
combined_df.rename(columns={'주말':'주말_공휴일'}, inplace=True)

# 주말 또는 공휴일을 1로, 평일을 0으로 설정
combined_df['주말_공휴일'] = combined_df.apply(lambda row:1 if row['주말_공휴일'] == 1 or pd.notna(row['공휴일']) else 0, axis=1)

combined_df['공휴일'] = combined_df.apply(lambda row:'주말' if row['주말_공휴일'] == 1 else ('평일' if pd.isna(row['공휴일']) else row['공휴일']), axis=1)

combined_df.head(35)

,일시,주말_공휴일,공휴일
0,2022-01-01,1,주말
1,2022-01-02,1,주말
2,2022-01-03,0,평일
3,2022-01-04,0,평일
4,2022-01-05,0,평일
5,2022-01-06,0,평일
6,2022-01-07,0,평일
7,2022-01-08,1,주말
8,2022-01-09,1,주말
9,2022-01-10,0,평일


In [54]:
combined_df.to_csv('combined_data.csv', index=False, encoding='utf-8')